<a href="https://colab.research.google.com/github/ashivashankars/CMPE255_Assignments/blob/main/tutorials/Tutorial%20-%20Multivariate_anomaly_detection_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Multivariate anomaly detection

In [1]:
!pip -q install gradio pandas scikit-learn --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 59.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


In [2]:
import io
import gradio as gr
import pandas as pd
from sklearn.ensemble import IsolationForest

In [3]:
import pandas as pd
sample = pd.DataFrame({
    "feature1":[58,59,62,59,87,60,61,90],
    "feature2":[77769,81799,74751,74373,17760,76000,77000,15000],
    "feature3":[0.7913,0.7910,0.7026,0.7656,0.3487,0.78,0.77,0.35],
    "feature4":[6559.82,5417.66,9258.99,7346.33,16869.50,7000,6900,17000]
})
sample.to_csv("/content/jewelry.csv", index=False)
print("Saved /content/jewelry.csv")


Saved /content/jewelry.csv


In [4]:
def detect_anomalies(file, contamination):
    # load CSV
    if file is None:
        return pd.DataFrame({"error":["Please upload a CSV"]})
    df = pd.read_csv(file.name)

    # keep only numeric columns (common pattern)
    num = df.select_dtypes(include="number")
    if num.empty:
        return pd.DataFrame({"error":["No numeric columns found"]})

    # fit model
    iso = IsolationForest(
        n_estimators=300,
        contamination=float(contamination),
        random_state=42
    )
    labels = iso.fit_predict(num)               # -1 = anomaly, 1 = normal
    scores = iso.decision_function(num)         # lower = more anomalous

    out = df.copy()
    out["anomaly"] = (labels == -1).astype(int) # 1 = anomaly
    out["anomaly_score"] = scores
    # sort by most anomalous first
    out = out.sort_values("anomaly_score")
    return out

In [5]:
with gr.Blocks(title="Anomaly Detector") as app:
    gr.Markdown("### CSV Anomaly Detection (Isolation Forest)\nUpload a CSV with numeric columns.")
    with gr.Row():
        file = gr.File(file_types=[".csv"], label="Upload CSV")
        contamination = gr.Slider(0.01, 0.2, value=0.05, step=0.01,
                                  label="Contamination (expected anomaly ratio)")
    btn = gr.Button("Detect anomalies")
    table = gr.Dataframe(wrap=True, label="Results (sorted by anomaly_score)")
    btn.click(detect_anomalies, inputs=[file, contamination], outputs=table)

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0620048662b17aecfa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
